# Conditional Power Outage Impact Model — Proof of Concept

This notebook presents a **rapid proof-of-concept (PoC)** for an **outage impact model** developed as a supporting component within a larger end-to-end outage risk and impact modeling system.

The objective of this PoC is **not to deliver a production-ready model**, but to quickly validate:
- whether a **weather-driven impact signal** exists once an outage has occurred, and  
- whether such a component can be reasonably integrated into a broader modeling framework.

Given this is exploratory phase of work, the implementation intentionally prioritizes **clarity, robustness, and reproducibility** over sophistication or exhaustive optimization.

---

## Problem Scope

The model focuses on **conditional impact estimation**:

> *Given that an outage has occurred on a county-day, estimate the number of affected customers.*

Key characteristics:
- Granularity: **county-day**
- Samples: limited (~200+ positive outage days)
- Target: total customers affected per county-day
- Inputs: aggregated weather and contextual features

This PoC **does not model outage occurrence probability** and should not be interpreted as estimating unconditional expected impact. For that purpose, please combine with the other occurrence probability model.

---

## Modeling Approach

Two modeling approaches are evaluated sequentially:

1. **Regularized linear regression (Ridge)**  
   - Serves as a transparent baseline  
   - Highlights the limitations of linear assumptions under small sample size and long-tailed impact distributions

2. **Gradient-boosted decision trees (XGBoost)**  
   - Introduced to capture non-linear weather–impact relationships  
   - Evaluated using time-based cross-validation  
   - Selected for final prediction due to consistently lower error metrics

To stabilize training:
- The target is modeled in **log(1 + customers)** space
- Simple imputation and one-hot encoding are applied
- Conservative model complexity is enforced to reduce overfitting risk

---

## Limitations of This PoC

This implementation makes several simplifying assumptions and has clear limitations:

- **Small sample size** limits statistical power and generalization
- **County-day aggregation** obscures event-level heterogeneity
- Weather features are coarse and may not align precisely with outage timing
- Model evaluation focuses on error reduction rather than probabilistic calibration or tail risk guarantees

These limitations are expected and acceptable at this stage, given the PoC’s role within a larger system.

---

## Planned Improvements

Future iterations would focus on:

- Integrating with **outage risk model** to estimate unconditional expected impact
- Moving from county-day to **event-level or feeder-level granularity**
- Refining weather feature windows around outage start times
- Improving tail modeling (e.g., high-impact day classification or quantile regression)
- Expanding training data across additional seasons and storm regimes

---

## Intended Use

The results of this PoC notebook are intended for:
- Architecture validation within a broader outage risk framework
- Early signal assessment and model feasibility testing
- Guiding prioritization of future data and modeling investments

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_log_error

## 1. Ridge regression

In [2]:
# -----------------------------
# 1) Read data
# -----------------------------
PATH = "cleaned_primary_table/conditional_impact_primary_table.csv"  # change if needed
df = pd.read_csv(PATH)

In [3]:
# -----------------------------
# 2) Basic cleanup / typing
# -----------------------------
# Ensure date is datetime and sort (critical for time-based CV)
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
else:
    raise ValueError("Expected a 'date' column in the dataset.")

df = df.sort_values("date").reset_index(drop=True)

# Target
TARGET = "total_customers"
if TARGET not in df.columns:
    raise ValueError(f"Expected target column '{TARGET}' in the dataset.")

# Optional: remove impossible/invalid targets (keep it simple)
df = df[df[TARGET].notna()].copy()
df = df[df[TARGET] >= 0].copy()

In [4]:
# -----------------------------
# 3) Define columns to drop / keep
# -----------------------------
# ID / non-feature columns to drop
DROP_COLS = [
    "county",           # identifier (high-cardinality); keep only if you explicitly want to model county fixed effects
    "county_code",      # identifier
    "code",             # if exists
    "start_time",       # if exists
    "start_date",       # if exists
    "date",             # used for splitting; not as numeric feature in this POC
]

DROP_COLS = [c for c in DROP_COLS if c in df.columns]

# Keep a small ID frame for outputs (what you want to see with predictions)
id_cols = [c for c in ["date", "county", "county_code"] if c in df.columns]
id_frame = df[id_cols].copy()

# Build feature matrix
X = df.drop(columns=DROP_COLS + [TARGET], errors="ignore")
y = df[TARGET].astype(float)

In [5]:
# -----------------------------
# 4) Identify categorical/numeric columns
# -----------------------------
# 'ar_intensity' is commonly categorical (e.g., AR2/AR3/AR4).
# We'll treat object/category dtype columns as categorical automatically.
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = [c for c in X.columns if c not in cat_cols]

In [6]:
# -----------------------------
# 5) Preprocessing + Model
# -----------------------------
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop",
)

# We model log1p(y) for stability on long-tailed customer impact
model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("ridge", Ridge(alpha=1.0, random_state=0)),
    ]
)


In [7]:
# -----------------------------
# 6) Time-based CV evaluation + out-of-fold predictions
# -----------------------------
tscv = TimeSeriesSplit(n_splits=4)

oof_pred = np.full(shape=len(df), fill_value=np.nan, dtype=float)

fold_metrics = []
for fold, (train_idx, test_idx) in enumerate(tscv.split(X), start=1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Fit on log1p(y)
    model.fit(X_train, np.log1p(y_train))

    # Predict, then invert transform
    pred_log = model.predict(X_test)
    pred = np.expm1(pred_log)
    pred = np.clip(pred, 0, None)

    oof_pred[test_idx] = pred

    # Metrics on original scale
    mae = mean_absolute_error(y_test, pred)

    # RMSLE: safe even if y has zeros; pred is clipped >= 0
    rmsle = np.sqrt(mean_squared_log_error(y_test, pred))

    fold_metrics.append({"fold": fold, "mae": mae, "rmsle": rmsle})

metrics_df = pd.DataFrame(fold_metrics)
print("\nTimeSeriesSplit CV metrics (original scale):")
print(metrics_df)
print("\nCV averages:")
print(metrics_df[["mae", "rmsle"]].mean().to_frame("mean"))


TimeSeriesSplit CV metrics (original scale):
   fold          mae     rmsle
0     1  3792.099426  1.676775
1     2  4356.355289  1.264583
2     3  1299.735307  0.958647
3     4  2225.209108  1.445127

CV averages:
              mean
mae    2918.349782
rmsle     1.336283


In [7]:
'''
# -----------------------------
# 7) Fit final model on all data + produce final predictions
# -----------------------------
model.fit(X, np.log1p(y))
final_pred = np.expm1(model.predict(X))
final_pred = np.clip(final_pred, 0, None)
# -----------------------------
# 8) Output predictions
# -----------------------------
out = id_frame.copy()
out["y_true_total_customers"] = y.values
out["pred_fit_all_total_customers"] = final_pred  # in-sample fitted predictions (for sanity check only)


OUT_PATH = "conditional_impact_predictions.csv"
out.to_csv(OUT_PATH, index=False)

print(f"\nSaved predictions to: {OUT_PATH}")
print("\nPreview:")
print(out.head(10))
'''


Saved predictions to: conditional_impact_predictions.csv

Preview:
        date           county  county_code  y_true_total_customers  \
0 2022-12-31           Amador         6005             2693.000000   
1 2022-12-31        El Dorado         6017             1321.333333   
2 2022-12-31             Napa         6055             1205.000000   
3 2022-12-31       Sacramento         6067              207.250000   
4 2022-12-31  San Luis Obispo         6079              512.666667   
5 2022-12-31           Solano         6095              365.625000   
6 2022-12-31           Sonoma         6097              585.666667   
7 2023-01-07      San Joaquin         6077              948.500000   
8 2023-01-07           Tehama         6103              392.583333   
9 2023-01-07         Monterey         6053             1660.985294   

   pred_fit_all_total_customers  
0                   1389.358955  
1                    936.226238  
2                    549.115033  
3                    461.

## 2. XGB

In [8]:
# ================================
# XGBoost POC – Data Preparation
# ================================

# Read data
df = pd.read_csv("cleaned_primary_table/conditional_impact_primary_table.csv")

# Date handling (for time-based CV)
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.sort_values("date").reset_index(drop=True)

# Target
TARGET = "total_customers"
y = df[TARGET].astype(float)

# Drop ID / non-feature columns (keep consistent with Ridge)
DROP_COLS = [
    "county",
    "county_code",
    "code",
    "start_time",
    "start_date",
    "date",
]

DROP_COLS = [c for c in DROP_COLS if c in df.columns]

X = df.drop(columns=DROP_COLS + [TARGET], errors="ignore")

# Log-transform target for stability
y_log = np.log1p(y)

print(f"Samples: {len(X)}, Features: {X.shape[1]}")


Samples: 224, Features: 33


In [9]:
# ================================
# XGBoost Experiment (CV)
# ================================

from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_log_error

from xgboost import XGBRegressor

# Identify feature types
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = [c for c in X.columns if c not in cat_cols]

# Preprocessing (intentionally simple)
preprocess = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    remainder="drop",
)

# Conservative XGB for small-sample regime
xgb = XGBRegressor(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,
    reg_lambda=1.0,
    objective="reg:squarederror",
    random_state=0,
    n_jobs=4,
)

model_xgb = Pipeline([
    ("preprocess", preprocess),
    ("xgb", xgb),
])

# Time-based CV
tscv = TimeSeriesSplit(n_splits=4)
oof_pred = np.full(len(X), np.nan)

for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train_log = y_log.iloc[train_idx]

    model_xgb.fit(X_train, y_train_log)

    pred_log = model_xgb.predict(X_test)
    pred = np.expm1(pred_log)
    pred = np.clip(pred, 0, None)

    oof_pred[test_idx] = pred

# Metrics (NO R²)
mask = ~np.isnan(oof_pred)

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, np.clip(y_pred, 0, None)))

print("=== XGBoost OOF Performance (Original Scale) ===")
print(f"OOF MAE   : {mean_absolute_error(y[mask], oof_pred[mask]):.2f}")
print(f"OOF RMSLE : {rmsle(y[mask], oof_pred[mask]):.4f}")


=== XGBoost OOF Performance (Original Scale) ===
OOF MAE   : 2193.64
OOF RMSLE : 1.1581


In [10]:
# ================================
# Final XGBoost Fit + Prediction
# ================================

# Fit on full dataset
model_xgb.fit(X, y_log)

# Final predictions (in-sample, for downstream use)
final_pred_log = model_xgb.predict(X)
final_pred = np.expm1(final_pred_log)
final_pred = np.clip(final_pred, 0, None)

# Output predictions with ground truth
out = df[["date", "county"]].copy()
out["actual_total_customers"] = df["total_customers"].values
out["pred_total_customers_xgb"] = final_pred

OUT_PATH = "conditional_impact_xgb_predictions.csv"
out.to_csv(OUT_PATH, index=False)

print(f"Saved final XGB predictions to: {OUT_PATH}")
print(out.head(10))


Saved final XGB predictions to: conditional_impact_xgb_predictions.csv
        date           county  actual_total_customers  \
0 2022-12-31           Amador             2693.000000   
1 2022-12-31        El Dorado             1321.333333   
2 2022-12-31             Napa             1205.000000   
3 2022-12-31       Sacramento              207.250000   
4 2022-12-31  San Luis Obispo              512.666667   
5 2022-12-31           Solano              365.625000   
6 2022-12-31           Sonoma              585.666667   
7 2023-01-07      San Joaquin              948.500000   
8 2023-01-07           Tehama              392.583333   
9 2023-01-07         Monterey             1660.985294   

   pred_total_customers_xgb  
0               1811.578613  
1               1393.087646  
2               1106.792114  
3                275.534180  
4                624.131531  
5                430.788513  
6                556.895447  
7                985.623047  
8                472.271362  
9

## 3. Merge with proability forecast

In [12]:
risk = pd.read_csv("risk_predictions_downloaded/risk_model_score_predictions.csv")
risk['date'] = pd.to_datetime(risk['date'])
impact = out.copy()

In [13]:
combined_results = pd.merge(risk, impact, left_on = ["date", "county_name"], right_on = ["date", "county"], how = "left")
combined_results.drop(columns=["county"], inplace=True)

In [14]:
combined_results[combined_results["risk score"] == 1]

,county_name,date,risk score,predicted_risk_score,actual_total_customers,pred_total_customers_xgb
92,Amador,2022-12-31,1.0,0.966251,2693.000000,1811.578613
278,El Dorado,2022-12-31,1.0,0.991054,1321.333333,1393.087646
867,Napa,2022-12-31,1.0,0.783266,1205.000000,1106.792114
1053,Sacramento,2022-12-31,1.0,0.600284,207.250000,275.534180
1239,San Luis Obispo,2022-12-31,1.0,0.653017,512.666667,624.131531
...,...,...,...,...,...,...
6057,Napa,2023-03-01,1.0,0.315728,218.000000,321.671021
6212,Riverside,2023-03-01,1.0,0.503726,1091.647059,1309.596191
6429,San Luis Obispo,2023-03-01,1.0,0.403509,515.111111,519.174194
6491,Santa Barbara,2023-03-01,1.0,0.435082,338.000000,368.260498


In [16]:
combined_results.to_csv("combined_risk_and_impact_predictions.csv", index = False)